In [1]:
import numpy as np
import pandas as pd
import os
import yaml
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
blue_shades = [
    "#1f4fd8",
    "#5779c9",
    "#203866",
    "#0055ff",
]
green_shades = [
    "#06d945",
    "#1da81d",
    "#1fc93e",
    "#1EA619",
]
orange_shades = [
    "#ff7f0e",
    "#ff9f3f",
    "#ff6f0e",
    "#ff5a00",
]
red_shade = "#ff0044"

vis_dict = {
    'M1': {'color':red_shade},
    'M2': {'color':green_shades[0]},
    'M3': {'color':green_shades[1]},
    'M4': {'color':green_shades[2]},
    'M5': {'color':green_shades[3]},
    'M6': {'color':blue_shades[0]},
    'M7': {'color':blue_shades[1]},
    'M8': {'color':blue_shades[2]},
    'M9': {'color':blue_shades[3]},
    'M10': {'color':orange_shades[0]},
    'M11': {'color':orange_shades[1]},
    'M12': {'color':orange_shades[2]},    
}
delimiter = '\t'

In [3]:

# Load run data
def load_run(run_file, run_id):
    """Load specific run configuration from YAML file."""
    with open(run_file, 'r') as f:
        runs = yaml.safe_load(f)
    
    run = runs.get(run_id)
    if run is None:
        raise KeyError(f"Run '{run_id}' not found in {run_file}")
    
    return run

In [4]:

def load_run(run_file, run_id):
    """Load specific run configuration from YAML file."""
    with open(run_file, 'r') as f:
        runs = yaml.safe_load(f)
    
    run = runs.get(run_id)
    if run is None:
        raise KeyError(f"Run '{run_id}' not found in {run_file}")
    
    return run

def load_device_status(device_status_file):
    """Load device status information from YAML file."""
    with open(device_status_file, 'r') as f:
        device_status = yaml.safe_load(f)
    
    return device_status

def get_contact_info(device_status, contact_name):
    """
    Get contact information including width and notes.
    
    Returns:
        dict with width and note (if any)
    """
    contacts = device_status.get('contacts', {})
    contact = contacts.get(contact_name, {})
    
    info = {
        'width': contact.get('width', 'N/A'),
        'note': contact.get('note', None),
        'hBN_thickness': contact.get('hBN_thickness', 'N/A'),
        'type': contact.get('type', 'N/A')
    }
    
    return info



In [5]:
def load_run_data(data_folder, run_id, drains):
    """
    Load all data files for a specific run and its drain channels.
    
    Args:
        data_folder: Path to folder containing .dat files
        run_id: Run identifier (e.g., 'MS2P6_run06')
        drains: List of drain identifiers (e.g., ['M7', 'M8', 'M9'])
    
    Returns:
        Dictionary mapping drain -> DataFrame
    """
    data = {}
    
    for drain in drains:
        # Get all files in folder
        all_files = os.listdir(data_folder)
        
        # Filter for this run and drain
        matching = [f for f in all_files 
                   if f.startswith(f"{run_id}_{drain}") 
                   and f.endswith('.dat')]
        
        if not matching:
            print(f"Warning: No data files found for {drain}")
            continue
        
        # Load files
        matching.sort()
        file_paths = [os.path.join(data_folder, f) for f in matching]
        
        if len(file_paths) == 1:
            data[drain] = pd.read_csv(file_paths[0], sep='\t')
        else:
            dfs = [pd.read_csv(f, sep='\t') for f in file_paths]
            data[drain] = pd.concat(dfs, ignore_index=True)
            print(f"Note: Concatenated {len(file_paths)} files for {drain}")
    
    return data

In [36]:
def get_plot_info(current_run):
    """
    Extract plotting information from run metadata.
    
    Returns:
        dict with column names, labels, units, and plot type
    """
    col_desc = current_run['dataframe_parameters']['columns_descriptions']
    data_cols = current_run['dataframe_parameters']['data_columns']
    
    # Get measurement label, treat 'none' as empty string
    meas_label = current_run.get('measurement_label', 'none')
    if meas_label == 'none':
        meas_label = ''
    
    plot_info = {
        'type': current_run['type'],
        'device': current_run['device'],
        'date': current_run['date'],
        'label': meas_label,
        'setup': current_run.get('setup', {}),
        'columns': {}
    }
    
    # Extract info for each column
    for col in data_cols:
        if col in col_desc:
            desc = col_desc[col]
            plot_info['columns'][col] = {
                'quantity': desc.get('quantity', col),
                'symbol': desc.get('symbol', ''),
                'unit': desc.get('unit', ''),
                'label': f"{desc.get('quantity', col)} ({desc.get('unit', '')})"
            }
    
    return plot_info

def get_column_by_quantity(plot_info, quantity):
    """Find column name by its quantity description."""
    for col_name, col_info in plot_info['columns'].items():
        if col_info['quantity'] == quantity:
            return col_name
    return None

In [46]:
def plot_measurement(data, current_run, device_status, vis_dict, run_id):
    """
    Universal plotter for both I-V and dI/dV measurements.
    Automatically adapts based on measurement type.
    
    Args:
        data: Dictionary of DataFrames (drain -> df)
        current_run: Run metadata from YAML
        device_status: Device status from YAML
        vis_dict: Color mapping for each drain
    """
    plot_info = get_plot_info(current_run)
    measurement_type = plot_info['type']
    
    # Get column names dynamically based on quantity
    voltage_col = get_column_by_quantity(plot_info, 'voltage')
    current_col = get_column_by_quantity(plot_info, 'current')
    
    # Validate basic columns exist
    if not all([voltage_col, current_col]):
        raise ValueError(f"Missing required columns. Found: V={voltage_col}, I={current_col}")
    
    # Check if this is a dI/dV measurement
    is_didv = measurement_type == 'dI/dV'
    
    if is_didv:
        # Get AC current columns for dI/dV
        iac_x_col = get_column_by_quantity(plot_info, 'current change, real part')
        iac_y_col = get_column_by_quantity(plot_info, 'current change, imaginary part')
        
        if not all([iac_x_col, iac_y_col]):
            raise ValueError(f"dI/dV measurement missing AC columns. Found: Iac_x={iac_x_col}, Iac_y={iac_y_col}")
        
        # Get modulation voltage
        V_ac = plot_info['setup'].get('ac_modulation_voltage', 'N/A')
        
        # Create figure with secondary y-axis
        fig = make_subplots(specs=[[{"secondary_y": True}]])
    else:
        # Simple figure for I-V
        fig = go.Figure()
    
    # Collect notes to display
    notes_list = []
    
    for drain, df in data.items():
        color = vis_dict.get(drain, {}).get('color', '#000000')
        
        # Get contact info
        contact_info = get_contact_info(device_status, drain)
        width = contact_info['width']
        
        # Create legend label with width
        if width != 'N/A':
            legend_base = f"{drain} (w={width} μm)"
        else:
            legend_base = drain
        
        # Collect note if exists
        if contact_info['note']:
            notes_list.append(f"{drain}: {contact_info['note']}")
        
        # Plot DC current
        if is_didv:
            # For dI/dV: current on left axis with label
            fig.add_trace(
                go.Scatter(
                    x=df[voltage_col],
                    y=df[current_col],
                    name=f"{legend_base} - I",
                    line=dict(color=color, width=2),
                    legendgroup=drain,
                ),
                secondary_y=False
            )
        else:
            # For I-V: simple plot
            fig.add_trace(
                go.Scatter(
                    x=df[voltage_col],
                    y=df[current_col],
                    name=legend_base,
                    line=dict(color=color, width=2),
                )
            )
        
        # Add dI/dV trace if applicable
        if is_didv:
            # Calculate dI/dV magnitude
            dIdV = np.sqrt(df[iac_x_col]**2 + df[iac_y_col]**2)
            
            # Normalize by modulation voltage if available
            if V_ac != 'N/A':
                try:
                    V_ac_value = float(V_ac)
                    dIdV = dIdV / V_ac_value
                    didv_label = f"{legend_base} - dI/dV"
                except (ValueError, TypeError):
                    didv_label = f"{legend_base} - dI/dV (unnormalized)"
            else:
                didv_label = f"{legend_base} - dI/dV (unnormalized)"
            
            # Plot dI/dV on right axis
            fig.add_trace(
                go.Scatter(
                    x=df[voltage_col],
                    y=dIdV,
                    name=didv_label,
                    line=dict(color=color, width=2, dash='dash'),
                    legendgroup=drain,
                ),
                secondary_y=True
            )
    
    # Get axis labels from metadata
    vdc_info = plot_info['columns'].get(voltage_col, {})
    idc_info = plot_info['columns'].get(current_col, {})
    
    # Build default labels from column info
    voltage_quantity = vdc_info.get('quantity', 'voltage')
    voltage_unit = vdc_info.get('unit', 'V')
    current_quantity = idc_info.get('quantity', 'current')
    current_unit = idc_info.get('unit', 'A')
    
    default_voltage_label = f"{voltage_quantity} ({voltage_unit})"
    default_current_label = f"{current_quantity} ({current_unit})"
    
    # Update x-axis
    fig.update_xaxes(title_text=vdc_info.get('label', default_voltage_label))
    
    # Update y-axis
    if is_didv:
        fig.update_yaxes(
            title_text=idc_info.get('label', default_current_label),
            secondary_y=False
        )
        
        # dI/dV axis label (right side)
        if V_ac != 'N/A':
            didv_unit = f"({current_unit}/{voltage_unit})"
        else:
            didv_unit = f"({current_unit}) - Vac: {V_ac}"
        
        fig.update_yaxes(
            title_text=f"dI/dV {didv_unit}",
            secondary_y=True
        )
    else:
        fig.update_yaxes(title_text=idc_info.get('label', default_current_label))
    
    # Title
    title = f"{run_id} {plot_info['label']}"
    
    # Build annotation text
    if is_didv:
        freq = plot_info['setup'].get('ac_modulation_frequency', 'N/A')
        annotations_text = f"f_mod: {freq} Hz"
        if V_ac != 'N/A':
            annotations_text += f", V_mod: {V_ac}"
    else:
        sweeps = current_run.get('measurement_parameters', {}).get('sweeps', 'N/A')
        v_range = current_run.get('measurement_parameters', {}).get('voltage_range', 'N/A')
        annotations_text = f"Sweeps: {sweeps}, Range: {v_range} V"
    
    # Add notes if any
    if notes_list:
        annotations_text += "<br>Notes: " + "; ".join(notes_list)
    
    fig.update_layout(
        title=title,        
        width=800,
        height=500,
        annotations=[
            dict(
                text=annotations_text,
                xref="paper", yref="paper",
                x=0.02, y=0.98,
                showarrow=False,
                align="left"
            )
        ]
    )
    
    fig.show()

# PLOTS

In [47]:
# Configuration
DEVICE_DIR = '/Volumes/DL_Share2/Boris/Devices/AFM Press [P]/MS2P/MS2P_6/05 Data'
device = 'MS2P_6'
RUN_ID = f'{device}_run01'

# Paths
data_folder = os.path.join(DEVICE_DIR, 'raw')
run_file = os.path.join(DEVICE_DIR, f'{device}_runs.yml')
device_status_file = os.path.join(DEVICE_DIR, f'{device}_device_status.yaml')

# Get current run
current_run = load_run(run_file, RUN_ID)
drains = current_run['acquisitions']['drain']

print(f"Loaded run: {RUN_ID}")
print(f"Device: {current_run.get('device')}")
print(f"Date: {current_run.get('date')}")

Loaded run: MS2P_6_run01
Device: MS2P_6
Date: 2026-01-15


In [48]:
# Load the run
current_run = load_run(run_file, RUN_ID)

# Load device status
device_status = load_device_status(device_status_file)

# Get drains
drains = current_run['acquisitions']['drain']

# Load the data
data = load_run_data(data_folder, RUN_ID, drains)

# Check what we loaded
for drain, df in data.items():
    print(f"{drain}: {len(df)} rows, columns: {list(df.columns)}")

# Plot (automatically detects I-V vs dI/dV)
plot_measurement(data, current_run, device_status, vis_dict, RUN_ID)

Note: Concatenated 2 files for M10
M2: 202 rows, columns: ['Vsample_V', 'Vsample_I']
M3: 202 rows, columns: ['Vsample_V', 'Vsample_I']
M4: 202 rows, columns: ['Vsample_V', 'Vsample_I']
M5: 202 rows, columns: ['Vsample_V', 'Vsample_I']
M6: 202 rows, columns: ['Vsample_V', 'Vsample_I']
M7: 205 rows, columns: ['Vsample_V', 'Vsample_I']
M8: 205 rows, columns: ['Vsample_V', 'Vsample_I']
M9: 202 rows, columns: ['Vsample_V', 'Vsample_I']
M10: 378 rows, columns: ['Vsample_V', 'Vsample_I']
M11: 202 rows, columns: ['Vsample_V', 'Vsample_I']
